In [2]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
hechos = pd.read_excel("datasets/homicidios.xlsx")
victimas = pd.read_excel("datasets/homicidios.xlsx", sheet_name=1)

# ETL 
### REVISION GENERAL DE LOS DATASETS
Evaluar cantidad de columnas y registros, tipos de variables, cantidad de nulos y duplicados, etc.

In [3]:
hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

In [4]:
hechos.head(3)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO


In [5]:
print(f"TABLA HECHOS \nDuplicados:  {hechos.duplicated().sum()} \nID duplicados:  {hechos['ID'].duplicated().sum()}")

TABLA HECHOS 
Duplicados:  0 
ID duplicados:  0


In [6]:
victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  717 non-null    object        
 6   VICTIMA              717 non-null    object        
 7   SEXO                 717 non-null    object        
 8   EDAD                 717 non-null    object        
 9   FECHA_FALLECIMIENTO  717 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 56.1+ KB


In [7]:
victimas.head()

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00


In [8]:
print(f"TABLA VICTIMAS \nDuplicados:  {victimas.duplicated().sum()}")

TABLA VICTIMAS 
Duplicados:  0


In [9]:
# Funcion para evaluar cuantos tipos de datos tienen las columnas de un dataframe (incluyendo nulos y SD en este caso)

def tiposDatos(df):
    dicc = {"Campo": [], "Tipo_de_dato": [], "Nulos": [], "SD": []}
    for i, columna in enumerate(df.columns):
        dicc["Campo"].append(columna)
        dicc["Tipo_de_dato"].append(df[columna].apply(type).unique())
        dicc["Nulos"].append(df[columna].isnull().sum())
        dicc["SD"].append(df[df[columna]=="SD"][columna].count())
    df_info = pd.DataFrame(dicc)
    return df_info

In [54]:
tiposDatos(hechos)

,Campo,Tipo_de_dato,Nulos,SD
0,ID,[<class 'str'>],0,0
1,N_VICTIMAS,[<class 'int'>],0,0
2,FECHA,[<class 'pandas._libs.tslibs.timestamps.Timest...,0,0
3,AAAA,[<class 'int'>],0,0
4,MM,[<class 'int'>],0,0
5,DD,[<class 'int'>],0,0
6,HORA,"[<class 'datetime.time'>, <class 'str'>, <clas...",0,1
7,HH,"[<class 'int'>, <class 'str'>]",0,1
8,LUGAR_DEL_HECHO,[<class 'str'>],0,1
9,TIPO_DE_CALLE,[<class 'str'>],0,0


In [38]:
tiposDatos(victimas)

,Campo,Tipo_de_dato,Nulos,SD
0,ID_hecho,[<class 'str'>],0,0
1,FECHA,[<class 'pandas._libs.tslibs.timestamps.Timest...,0,0
2,AAAA,[<class 'int'>],0,0
3,MM,[<class 'int'>],0,0
4,DD,[<class 'int'>],0,0
5,ROL,[<class 'str'>],0,11
6,VICTIMA,[<class 'str'>],0,9
7,SEXO,[<class 'str'>],0,6
8,EDAD,"[<class 'int'>, <class 'str'>]",0,53
9,FECHA_FALLECIMIENTO,"[<class 'datetime.datetime'>, <class 'str'>]",0,68


In [12]:
# CORREGIR ERROR EN REGISTRO 64
hechos['FECHA'] = hechos['FECHA'].dt.strftime('%Y-%m-%d')  # convertir formato datetime a str para realizar el cambio
hechos.loc[64,'FECHA'] = '2016-07-03'  # cambiar el día 2 por 3
hechos['FECHA'] = pd.to_datetime(hechos['FECHA'])  # volver a convertir al formato datetime
hechos.loc[hechos['ID'] == "2016-0087"]  # verificar

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
64,2016-0087,1,2016-07-03,2016,7,3,00:10:00,0,AUTOPISTA 1 SUR PTE ARTURO FRONDIZI Y AV CASEROS,AUTOPISTA,...,NaN,NaN,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI y C...,1,Point (107762.62066736 100018.90176187),-58.37864583,-34.62907067,MOTO-OBJETO FIJO,MOTO,OBJETO FIJO


In [13]:
hechos["FECHA"] = pd.to_datetime(hechos["FECHA"])

AHORA TRABAJAR CON LAS COLUMNAS HORA Y HH

In [14]:
hechos[hechos['HH']=="SD"]

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
518,2019-0103,1,2019-12-18,2019,12,18,SD,SD,"PAZ, GRAL. AV. Y GRIVEO",GRAL PAZ,...,NaN,GRIVEO,"PAZ, GRAL. AV. y GRIVEO",11,Point (94643.11254058 103831.57115061),-58.52169422,-34.59471640,MOTO-MOTO,MOTO,MOTO


Se decide imputar el dato faltante calculando el promedio del total de las horas (en los campos HORA y HH)

In [15]:
hora_promedio = round(pd.to_numeric(hechos["HH"], errors="coerce").mean())

In [16]:
# hechos['HORA'] = hechos['HORA'].replace({'SD': hora_promedio})
hechos.loc[518,"HH"] = hora_promedio

Borrar el campo HORA (y cambiar el nombre de HH a "hora")

In [17]:
del hechos['HORA']

In [18]:
# RENOMBRAR HH A HORA

hechos.rename(columns={'HH':'HORA'}, inplace=True)

In [19]:
# CAMBIAR A TIPO INT

hechos['HORA'] = hechos['HORA'].astype(np.int64)

RENOMBRAR OTRAS COLUMNAS

In [20]:
hechos.rename(columns={'ID': 'ID_HECHO', 'AAAA':'ANIO', 'MM':'MES', 'DD':'DIA', 'Calle':'CALLE', 'Cruce':'CRUCE', 'pos X': 'LONGITUD', 'pos Y': 'LATITUD'}, inplace=True)

AGREGAR COLUMNA "FRANJA_HORARIA" A PARTIR DE "HORA"

In [26]:
Franja = ["MANIANA_6-10","MEDIODIA_11-15","TARDE_16-20","NOCHE_21-1","MADRUGADA_2-5"]
# Franja = ["MAÑANA","MEDIODIA","TARDE","NOCHE","MADRUGADA"]

horas = []
for ind, hora in enumerate(hechos["HORA"]):
    if hora in range(6,11):
        horas.append(Franja[0])
    elif hora in range(11,16):
        horas.append(Franja[1])
    elif hora in range(16,21):
        horas.append(Franja[2])
    elif hora in range(21,24):
        horas.append(Franja[3])
    elif hora in range(0,2):
        horas.append(Franja[3])
    elif hora in range(2,6):
        horas.append(Franja[4])

hechos["FRANJA_HORARIA"] = horas

In [22]:
# REORDENAR LAS COLUMNAS
hechos = hechos.iloc[:,[0,1,2,3,4,5,6,20,7,8,9,10,11,12,13,14,15,16,17,18,19]]

In [28]:
hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID_HECHO               696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   ANIO                   696 non-null    int64         
 4   MES                    696 non-null    int64         
 5   DIA                    696 non-null    int64         
 6   HORA                   696 non-null    int64         
 7   FRANJA_HORARIA         696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  CALLE                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  CRUCE                  525 non-null    object        
 13  Direc

CREAR COLUMNA "DIA_SEMANA" A PARTIR DE "FECHA"

In [29]:
hechos['DIA_SEMANA'] = hechos['FECHA'].dt.day_name()

In [30]:
# REORDENAR LAS COLUMNAS

hechos = hechos.iloc[:,[0,1,2,21,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]]

TRADUCIR AL ESPAÑOL LOS DIAS DE LA SEMANA

In [31]:
for ind, dia in enumerate(hechos["DIA_SEMANA"]):
    if dia == "Monday":
        hechos.loc[ind,"DIA_SEMANA"] = "LUNES"
    if dia == "Tuesday":
        hechos.loc[ind,"DIA_SEMANA"] = "MARTES"
    if dia == "Wednesday":
        hechos.loc[ind,"DIA_SEMANA"] = "MIERCOLES"
    if dia == "Thursday":
        hechos.loc[ind,"DIA_SEMANA"] = "JUEVES"
    if dia == "Friday":
        hechos.loc[ind,"DIA_SEMANA"] = "VIERNES"
    if dia == "Saturday":
        hechos.loc[ind,"DIA_SEMANA"] = "SABADO"
    if dia == "Sunday":
        hechos.loc[ind,"DIA_SEMANA"] = "DOMINGO"

In [32]:
hechos.head(3)

,ID_HECHO,N_VICTIMAS,FECHA,DIA_SEMANA,ANIO,MES,DIA,HORA,FRANJA_HORARIA,LUGAR_DEL_HECHO,...,Altura,CRUCE,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,VIERNES,2016,1,1,4,MADRUGADA_2-5,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,SABADO,2016,1,2,1,NOCHE_21-1,AV GRAL PAZ Y AV DE LOS CORRALES,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,DOMINGO,2016,1,3,7,MANIANA_6-10,AV ENTRE RIOS 2034,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO


TRABAJAR CON LA COLUMNA CALLE

In [33]:
# RELLENAR LOS NULOS CON EL VALOR "SD"

hechos = hechos.fillna(value={"CALLE": "SD"})

In [34]:
hechos[hechos["CALLE"]=="SD"]

,ID_HECHO,N_VICTIMAS,FECHA,DIA_SEMANA,ANIO,MES,DIA,HORA,FRANJA_HORARIA,LUGAR_DEL_HECHO,...,Altura,CRUCE,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
119,2016-0151,1,2016-11-18,VIERNES,2016,11,18,20,TARDE_16-20,SD,...,NaN,NaN,NaN,0,Point (. .),.,.,PEATON-SD,PEATON,SD


In [35]:
# BUSCAR NOMBRES DE CALLE QUE CONTENGAN EL CARACTER "?"

indices = []
for ind, calle in enumerate(hechos["CALLE"]):
    if "?" in calle:
        indices.append(ind)
        print(ind, calle)

50 MAGARI?OS CERVANTES, A.
134 NU?EZ
173 CASTA?ARES AV.
241 ALBARI?O
370 AUTOPISTA PRESIDENTE H?CTOR J. C?MPORA
377 CASTA?ARES AV.
386 MU?ECAS
419 CASTA?ARES AV.
429 CASTA?ARES AV.
524 RODRIGUEZ PE?A


In [36]:
# COMPARAR CON "LUGAR_DEL_HECHO" PARA ARREGLAR

hechos.loc[indices,"LUGAR_DEL_HECHO"]

50                   MAGARIÑOS CERVANTES 3900
134                 NUÑEZ Y AV RICARDO BALBIN
173                  AV. CASTAÑARES Y LAUTARO
241                  ALBARIÑO Y AV. ARGENTINA
370    AU Héctor J. Cámpora y Roca, Cnel. Av.
377                 Av. Castañares y Bonorino
386                         Muñecas y Serrano
419       CASTAÑARES AV. Y MORENO, PERITO AV.
429        CASTAÑARES AV. Y GORDILLO, TIMOTEO
524            RODRIGUEZ PEÑA Y RIVADAVIA AV.
Name: LUGAR_DEL_HECHO, dtype: object

In [37]:
# ARREGLAR EL CARACTER ? EN LA COLUMNA CALLE

hechos.loc[50,"CALLE"] = "MAGARIÑOS CERVANTES, A"
hechos.loc[134,"CALLE"] = "NUÑEZ"
hechos.loc[173,"CALLE"] = "CASTAÑARES, AV."
hechos.loc[241,"CALLE"] = "ALBARIÑO"
hechos.loc[370,"CALLE"] = "HECTOR J. CAMPORA, AU."
hechos.loc[377,"CALLE"] = "CASTAÑARES, AV."
hechos.loc[386,"CALLE"] = "MUÑECAS"
hechos.loc[419,"CALLE"] = "CASTAÑARES, AV."
hechos.loc[429,"CALLE"] = "CASTAÑARES, AV."
hechos.loc[524,"CALLE"] = "RODRIGUEZ PEÑA"

In [38]:
# CORROBORAR
hechos.loc[indices,["CALLE"]]

,CALLE
50,"MAGARIÑOS CERVANTES, A"
134,NUÑEZ
173,"CASTAÑARES, AV."
241,ALBARIÑO
370,"HECTOR J. CAMPORA, AU."
377,"CASTAÑARES, AV."
386,MUÑECAS
419,"CASTAÑARES, AV."
429,"CASTAÑARES, AV."
524,RODRIGUEZ PEÑA


BORRAR COLUMNA ALTURA

In [39]:
del hechos["Altura"]

TRABAJAR CON LA COLUMNA CRUCE

In [40]:
# RELLENAR LOS NULOS CON EL VALOR N/A (NO APLICA)

hechos = hechos.fillna(value={"CRUCE":"N/A"})
# hechos["CRUCE"] = hechos["CRUCE"].fillna(value="N/A") 

In [41]:
# REPETIR LO MISMO QUE EN CALLE (CORREGIR CARACTERES ?)

indices = []
for ind, cruce in enumerate(hechos["CRUCE"]):
    if "?" in cruce:
        indices.append(ind)
        print(ind, cruce)

4 SAENZ PE?A, LUIS, PRES.
65 NU?EZ
209 RODRIGUEZ PE?A
221 NU?EZ
244 CA?ADA DE GOMEZ
331 HUMBERTO 1?
445 ORO?O, NICASIO
469 CASTA?ARES AV.
588 SAENZ PE?A, LUIS, PRES.


In [42]:
hechos.loc[indices,"LUGAR_DEL_HECHO"]

4        AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA 
65                                 CERETTI Y NUÑEZ
209            AV. DEL LIBERTADOR Y RODRIGUEZ PEÑA
221                            AV. CABILDO Y NUÑEZ
244                  AV. RIESTRA Y CAÑADA DE GOMEZ
331             Combate De Los Pozos y Humberto 1°
445            JUSTO, JUAN B. AV. y OROÑO, NICASIO
469                  ASTURIAS AV. Y CASTAÑARES AV.
588    INDEPENDENCIA AV. Y SAENZ PEÑA, LUIS, PRES.
Name: LUGAR_DEL_HECHO, dtype: object

In [43]:
hechos.loc[4,"CRUCE"] = "SAENZ PEÑA, LUIS, PRES."
hechos.loc[65,"CRUCE"] = "NUÑEZ"
hechos.loc[209,"CRUCE"] = "RODRIGUEZ PEÑA"
hechos.loc[221,"CRUCE"] = "NUÑEZ"
hechos.loc[244,"CRUCE"] = "CAÑADA DE GOMEZ"
hechos.loc[331,"CRUCE"] = "HUMBERTO 1º"
hechos.loc[445,"CRUCE"] = "OROÑO, NICASIO"
hechos.loc[469,"CRUCE"] = "CASTAÑARES AV."
hechos.loc[588,"CRUCE"] = "SAENZ PEÑA, LUIS, PRES."

TRABAJAR EN LA COLUMNA DIRECCION_NORMALIZADA

In [44]:
# COMPLETAR LOS DATOS FALTANTES EN DIRECCION_NORMALIZADA CON LOS DE LA COLUMNA "LUGAR_DEL_HECHO"

indices = []
for ind, direc in enumerate(hechos["Dirección Normalizada"]):
    if pd.isna(direc):
        indices.append(ind)
        print(ind, hechos.loc[ind, 'LUGAR_DEL_HECHO'])

38 AUTOPISTA LUGONES PK 10000
106 AU BUENOS AIRES - LA PLATA KM. 4
119 SD
180 AU PERITO MORENO Y RAMAL ENLACE AU1/AU6
181 AU DELLEPIANE 2400
313 AUTOPISTA LUGONES KM 4.7
546 LUGONES, LEOPOLDO AV. KM 6,1
621 AU BUENOS AIRES LA PLATA  KM 4,5


In [45]:
hechos.loc[38,"Dirección Normalizada"] = "LUGONES km 10000, AU."
hechos.loc[106,"Dirección Normalizada"] = "BUENOS AIRES - LA PLATA KM. 4, AU."
hechos.loc[119,"Dirección Normalizada"] = "SD"
hechos.loc[180,"Dirección Normalizada"] = "PERITO MORENO Y RAMAL ENLACE AU1/AU6, AU."
hechos.loc[181,"Dirección Normalizada"] = "DELLEPIANE 2400, AU."
hechos.loc[313,"Dirección Normalizada"] = "LUGONES KM 4.7, AU."
hechos.loc[546,"Dirección Normalizada"] = "LUGONES, LEOPOLDO KM 6,1, AV."
hechos.loc[621,"Dirección Normalizada"] = "BUENOS AIRES - LA PLATA KM 4,5, AU."

In [46]:
# Borrar LUGAR_DEL_HECHO y cambiar nombre a Dirección Normalizada

del hechos["LUGAR_DEL_HECHO"]
hechos.rename(columns={"Dirección Normalizada":"LUGAR_DEL_HECHO"},inplace=True)

In [47]:
# Corregir algunos caracteres de la columna LUGAR_DEL_HECHO (antes Dirección Normalizada)

indices = []
for ind, direc in enumerate(hechos["LUGAR_DEL_HECHO"]):
    if "Ã" in direc:
        indices.append(ind)
    if "Â" in direc:
        indices.append(ind)

# Ã‘ - Ñ  ;  Ã‰ - E  ;  Ã - A  ;  Â° - º

In [48]:
hechos.loc[indices,"LUGAR_DEL_HECHO"]

4                SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.
50                         MAGARIÃ‘OS CERVANTES, A. 3900
65                                      CERETTI y NUÃ‘EZ
134                    NUÃ‘EZ y BALBIN, RICARDO, DR. AV.
173                            CASTAÃ‘ARES AV. y LAUTARO
209                 DEL LIBERTADOR AV. y RODRIGUEZ PEÃ‘A
221                                 CABILDO AV. y NUÃ‘EZ
241                            ALBARIÃ‘O y ARGENTINA AV.
244                       RIESTRA AV. y CAÃ‘ADA DE GOMEZ
331                  COMBATE DE LOS POZOS y HUMBERTO 1Â°
355                 CASTAÃ‘ARES AV. y RIVERA INDARTE AV.
370    AUTOPISTA PRESIDENTE HÃ‰CTOR J. CÃMPORA y ROC...
377       CASTAÃ‘ARES AV. y BONORINO, ESTEBAN, CNEL. AV.
386                                   MUÃ‘ECAS y SERRANO
419                 CASTAÃ‘ARES AV. y MORENO, PERITO AV.
429                  CASTAÃ‘ARES AV. y GORDILLO, TIMOTEO
445                 JUSTO, JUAN B. AV. y OROÃ‘O, NICASIO
469                       ASTUR

In [49]:
hechos.loc[4,"LUGAR_DEL_HECHO"] = "SAN JUAN AV. y SAENZ PEÑA, LUIS, PRES."
hechos.loc[50,"LUGAR_DEL_HECHO"] = "MAGARIÑOS CERVANTES, A. 3900"
hechos.loc[65,"LUGAR_DEL_HECHO"] = "CERETTI y NUÑEZ"
hechos.loc[134,"LUGAR_DEL_HECHO"] = "NUÑEZ y BALBIN, RICARDO, DR. AV."
hechos.loc[173,"LUGAR_DEL_HECHO"] = "CASTAÑARES AV. y LAUTARO"
hechos.loc[209,"LUGAR_DEL_HECHO"] = "DEL LIBERTADOR AV. y RODRIGUEZ PEÑA"
hechos.loc[221,"LUGAR_DEL_HECHO"] = "CABILDO AV. y NUÑEZ"
hechos.loc[241,"LUGAR_DEL_HECHO"] = "ALBARIÑO y ARGENTINA AV."
hechos.loc[244,"LUGAR_DEL_HECHO"] = "RIESTRA AV. y CAÑADA DE GOMEZ"
hechos.loc[331,"LUGAR_DEL_HECHO"] = "COMBATE DE LOS POZOS y HUMBERTO 1º"
hechos.loc[355,"LUGAR_DEL_HECHO"] = "CASTAÑARES AV. y RIVERA INDARTE AV."
hechos.loc[370,"LUGAR_DEL_HECHO"] = "PRESIDENTE HECTOR J. CAMPORA, AU. y ROCA"
hechos.loc[377,"LUGAR_DEL_HECHO"] = "CASTAÑARES AV. y BONORINO, ESTEBAN, CNEL. AV."
hechos.loc[386,"LUGAR_DEL_HECHO"] = "MUÑECAS y SERRANO"
hechos.loc[419,"LUGAR_DEL_HECHO"] = "CASTAÑARES AV. y MORENO, PERITO AV."
hechos.loc[429,"LUGAR_DEL_HECHO"] = "CASTAÑARES AV. y GORDILLO, TIMOTEO"
hechos.loc[445,"LUGAR_DEL_HECHO"] = "JUSTO, JUAN B. AV. y OROÑO, NICASIO"
hechos.loc[469,"LUGAR_DEL_HECHO"] = "ASTURIAS AV. y CASTAÑARES AV."
hechos.loc[524,"LUGAR_DEL_HECHO"] = "RODRIGUEZ PEÑA y RIVADAVIA AV."
hechos.loc[567,"LUGAR_DEL_HECHO"] = "PRESIDENTE HECTOR J. CAMPORA, AU. y LAC."
hechos.loc[588,"LUGAR_DEL_HECHO"] = "INDEPENDENCIA AV. y SAENZ PEÑA, LUIS, PRES."

Borrar columna XY (CABA)

In [50]:
del hechos["XY (CABA)"]

In [51]:
hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   ID_HECHO         696 non-null    object        
 1   N_VICTIMAS       696 non-null    int64         
 2   FECHA            696 non-null    datetime64[ns]
 3   DIA_SEMANA       696 non-null    object        
 4   ANIO             696 non-null    int64         
 5   MES              696 non-null    int64         
 6   DIA              696 non-null    int64         
 7   HORA             696 non-null    int64         
 8   FRANJA_HORARIA   696 non-null    object        
 9   TIPO_DE_CALLE    696 non-null    object        
 10  CALLE            696 non-null    object        
 11  CRUCE            696 non-null    object        
 12  LUGAR_DEL_HECHO  696 non-null    object        
 13  COMUNA           696 non-null    int64         
 14  pos x            696 non-null    object   

TRABAJAR CON COLUMNAS LONGITUD Y LATITUD

In [52]:
hechos.rename(columns={"pos x":"LONGITUD","pos y":"LATITUD"},inplace=True)

In [53]:
# Evaluar registros donde Longitud y Latitud no tenga datos (con un ".") para verificar la posibilidad de identificarlos

indices = []
for ind, direc in enumerate(hechos["LONGITUD"]):
    if direc == ".":
        indices.append(ind)
hechos.loc[indices,["LUGAR_DEL_HECHO","CALLE","CRUCE","TIPO_DE_CALLE","COMUNA"]]

,LUGAR_DEL_HECHO,CALLE,CRUCE,TIPO_DE_CALLE,COMUNA
38,"LUGONES km 10000, AU.","LUGONES, LEOPOLDO AV.",N/A,AUTOPISTA,13
106,"BUENOS AIRES - LA PLATA KM. 4, AU.",AUTOPISTA BUENOS AIRES - LA PLATA,N/A,AUTOPISTA,4
119,SD,SD,N/A,CALLE,0
139,AUTOPISTA 25 DE MAYO,AUTOPISTA 25 DE MAYO,N/A,AUTOPISTA,0
176,"LUGONES, LEOPOLDO AV.","LUGONES, LEOPOLDO AV.",N/A,GRAL PAZ,14
180,"PERITO MORENO Y RAMAL ENLACE AU1/AU6, AU.",AUTOPISTA PERITO MORENO,N/A,AUTOPISTA,9
181,"DELLEPIANE 2400, AU.",AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,N/A,AUTOPISTA,7
256,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,N/A,AUTOPISTA,4
313,"LUGONES KM 4.7, AU.","LUGONES, LEOPOLDO AV.",N/A,AUTOPISTA,14
546,"LUGONES, LEOPOLDO KM 6,1, AV.","LUGONES, LEOPOLDO AV.",N/A,AUTOPISTA,14


Intentar ubicar en el mapa las siguientes direcciones para imputar una LATITUD y LONGITUD aproximadas:
* AUTOPISTA LUGONES, LEOPOLDO - COMUNAS: 13 (1), 14 (3, en uno le pusieron GRAL PAZ)
* AUTOPISTA BUENOS AIRES / LA PLATA (km 4 y km 4,5) - COMUNAS: 4 (2)
* AUTOPISTA 25 DE MAYO ?
* AUTOPISTA PERITO MORENO Y RAMAL ENLACE (AU1/AU6) - COMUNAS: 9
* AUTOPISTA DELLEPIANE LUIS 2400 - COMUNA: 7
* AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONIDZI - COMUNA: 4
* MURGUIONDO - COMUNA: 9


TRABAJAR CON VICTIMA Y ACUSADO

In [54]:
# CORREGIR EL ERROR DEL OBJETO FIJO EN VICTIMA (EL CUAL DEBE PASAR A ACUSADO Y VICEVERSA)

hechos[hechos["VICTIMA"]=="OBJETO FIJO"]

,ID_HECHO,N_VICTIMAS,FECHA,DIA_SEMANA,ANIO,MES,DIA,HORA,FRANJA_HORARIA,TIPO_DE_CALLE,CALLE,CRUCE,LUGAR_DEL_HECHO,COMUNA,LONGITUD,LATITUD,PARTICIPANTES,VICTIMA,ACUSADO
230,2017-0108,2,2017-09-02,SABADO,2017,9,2,4,MADRUGADA_2-5,GRAL PAZ,"PAZ, GRAL. AV.",MACHAIN,"PAZ, GRAL. AV. y MACHAIN",12,-58.49491054,-34.54795581,AUTO-OBJETO FIJO,OBJETO FIJO,AUTO


In [55]:
hechos.loc[230,"VICTIMA"] = "AUTO"
hechos.loc[230,"ACUSADO"] = "OBJETO FIJO"

In [56]:
victimas

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...
712,2021-0092,2021-12-12,2021,12,12,PEATON,PEATON,FEMENINO,50,2021-12-12 00:00:00
713,2021-0093,2021-12-13,2021,12,13,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,18,2021-12-18 00:00:00
714,2021-0094,2021-12-20,2021,12,20,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,43,2021-12-20 00:00:00
715,2021-0095,2021-12-30,2021,12,30,CONDUCTOR,MOTO,MASCULINO,27,2022-01-02 00:00:00


In [57]:
# RENOMBRAR ID_hecho por ID_HECHO

victimas.rename(columns={"ID_hecho":"ID_HECHO"}, inplace=True)

BORRAR COLUMNAS AAAA, MM, DD (compartidas con el dataset "hechos")

In [58]:
del victimas["AAAA"]
del victimas["MM"]
del victimas["DD"]

CREAR COLUMNA ID_VICTIMA (agregando un valor a ID_HECHO)

In [59]:
victimas["ID_VICTIMA"]=range(1,len(victimas)+1)

In [60]:
victimas["ID_VICTIMA"] = victimas["ID_VICTIMA"].astype(str)

In [61]:
victimas["ID_VICTIMA"] = victimas["ID_HECHO"]+"-"+victimas["ID_VICTIMA"].astype(str)

In [62]:
victimas = victimas.iloc[:, [7,0,1,2,3,4,5,6]]

In [63]:
victimas

,ID_VICTIMA,ID_HECHO,FECHA,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001-1,2016-0001,2016-01-01,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002-2,2016-0002,2016-01-02,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003-3,2016-0003,2016-01-03,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004-4,2016-0004,2016-01-10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005-5,2016-0005,2016-01-21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00
...,...,...,...,...,...,...,...,...
712,2021-0092-713,2021-0092,2021-12-12,PEATON,PEATON,FEMENINO,50,2021-12-12 00:00:00
713,2021-0093-714,2021-0093,2021-12-13,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,18,2021-12-18 00:00:00
714,2021-0094-715,2021-0094,2021-12-20,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,43,2021-12-20 00:00:00
715,2021-0095-716,2021-0095,2021-12-30,CONDUCTOR,MOTO,MASCULINO,27,2022-01-02 00:00:00


In [64]:
victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_VICTIMA           717 non-null    object        
 1   ID_HECHO             717 non-null    object        
 2   FECHA                717 non-null    datetime64[ns]
 3   ROL                  717 non-null    object        
 4   VICTIMA              717 non-null    object        
 5   SEXO                 717 non-null    object        
 6   EDAD                 717 non-null    object        
 7   FECHA_FALLECIMIENTO  717 non-null    object        
dtypes: datetime64[ns](1), object(7)
memory usage: 44.9+ KB


In [65]:
tiposDatos(victimas)

,Campo,Tipo_de_dato,Nulos,SD
0,ID_VICTIMA,[<class 'str'>],0,0
1,ID_HECHO,[<class 'str'>],0,0
2,FECHA,[<class 'pandas._libs.tslibs.timestamps.Timest...,0,0
3,ROL,[<class 'str'>],0,11
4,VICTIMA,[<class 'str'>],0,9
5,SEXO,[<class 'str'>],0,6
6,EDAD,"[<class 'int'>, <class 'str'>]",0,53
7,FECHA_FALLECIMIENTO,"[<class 'datetime.datetime'>, <class 'str'>]",0,68


CREAR COLUMNA DE RANGO_ETAREO A PARTIR DE EDAD

In [68]:
rangos_edad = ["<18", "18_33", "34_49", "50_65", ">65"]

edades = []
for ind, edad in enumerate(victimas["EDAD"]):
    if edad == "SD":
        edades.append("SD")
    if type(edad)==int:
        if edad < 18:
            edades.append(rangos_edad[0])
        elif edad in range(18,34):
            edades.append(rangos_edad[1])
        elif edad in range(34,50):
            edades.append(rangos_edad[2])
        elif edad in range(50,65):
            edades.append(rangos_edad[3])
        elif edad >= 65:
            edades.append(rangos_edad[4])

victimas.loc[:, "RANGO_ETAREO"] = edades

C:\Users\Intel\AppData\Local\Temp\ipykernel_11576\2497224185.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  victimas.loc[:, "RANGO_ETAREO"] = edades


In [69]:
victimas.head(3)

,ID_VICTIMA,ID_HECHO,FECHA,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO,RANGO_ETAREO
0,2016-0001-1,2016-0001,2016-01-01,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00,18_33
1,2016-0002-2,2016-0002,2016-01-02,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00,>65
2,2016-0003-3,2016-0003,2016-01-03,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00,18_33


In [70]:
victimas["RANGO_ETAREO"].value_counts()

RANGO_ETAREO
18_33    264
34_49    167
>65      112
50_65     98
SD        53
<18       23
Name: count, dtype: int64

In [71]:
victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_VICTIMA           717 non-null    object        
 1   ID_HECHO             717 non-null    object        
 2   FECHA                717 non-null    datetime64[ns]
 3   ROL                  717 non-null    object        
 4   VICTIMA              717 non-null    object        
 5   SEXO                 717 non-null    object        
 6   EDAD                 717 non-null    object        
 7   FECHA_FALLECIMIENTO  717 non-null    object        
 8   RANGO_ETAREO         717 non-null    object        
dtypes: datetime64[ns](1), object(8)
memory usage: 50.5+ KB


In [72]:
# REORDENAR COLUMNAS

victimas = victimas.iloc[:,[0,1,2,3,4,5,6,8,7]]

In [73]:
victimas.head()

,ID_VICTIMA,ID_HECHO,FECHA,ROL,VICTIMA,SEXO,EDAD,RANGO_ETAREO,FECHA_FALLECIMIENTO
0,2016-0001-1,2016-0001,2016-01-01,CONDUCTOR,MOTO,MASCULINO,19,18_33,2016-01-01 00:00:00
1,2016-0002-2,2016-0002,2016-01-02,CONDUCTOR,AUTO,MASCULINO,70,>65,2016-01-02 00:00:00
2,2016-0003-3,2016-0003,2016-01-03,CONDUCTOR,MOTO,MASCULINO,30,18_33,2016-01-03 00:00:00
3,2016-0004-4,2016-0004,2016-01-10,CONDUCTOR,MOTO,MASCULINO,18,18_33,SD
4,2016-0005-5,2016-0005,2016-01-21,CONDUCTOR,MOTO,MASCULINO,29,18_33,2016-02-01 00:00:00


In [ ]:
# victimas["EDAD"] = pd.to_numeric(victimas["EDAD"], errors="coerce")

In [75]:
victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_VICTIMA           717 non-null    object        
 1   ID_HECHO             717 non-null    object        
 2   FECHA                717 non-null    datetime64[ns]
 3   ROL                  717 non-null    object        
 4   VICTIMA              717 non-null    object        
 5   SEXO                 717 non-null    object        
 6   EDAD                 717 non-null    object        
 7   RANGO_ETAREO         717 non-null    object        
 8   FECHA_FALLECIMIENTO  717 non-null    object        
dtypes: datetime64[ns](1), object(8)
memory usage: 50.5+ KB


In [74]:
hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   ID_HECHO         696 non-null    object        
 1   N_VICTIMAS       696 non-null    int64         
 2   FECHA            696 non-null    datetime64[ns]
 3   DIA_SEMANA       696 non-null    object        
 4   ANIO             696 non-null    int64         
 5   MES              696 non-null    int64         
 6   DIA              696 non-null    int64         
 7   HORA             696 non-null    int64         
 8   FRANJA_HORARIA   696 non-null    object        
 9   TIPO_DE_CALLE    696 non-null    object        
 10  CALLE            696 non-null    object        
 11  CRUCE            696 non-null    object        
 12  LUGAR_DEL_HECHO  696 non-null    object        
 13  COMUNA           696 non-null    int64         
 14  LONGITUD         696 non-null    object   

In [9]:
victimas.to_csv('datasets/victimas.csv', index=False)

In [10]:
hechos.to_csv('datasets/hechos.csv', index=False)

In [12]:
del victimas["FECHA"]
del victimas["FECHA_FALLECIMIENTO"]
del victimas["VICTIMA"]

In [17]:
del hechos["N_VICTIMAS"]
del hechos["ANIO"]
del hechos["MES"]
del hechos["DIA"]
del hechos["DIA_SEMANA"]
del hechos["FRANJA_HORARIA"]

In [ ]:
# Pasar nombres de titulos a minusculas
# TIPO_DE_CALLE : tipo_via
# CALLE : nombre_via
# LUGAR_DEL_HECHO (podría borrarse) ?? pero evaluar qeu en CRUCE hay varios nulos ??
# PARTICIPANTES (podría borrarse) ?? es la combinación de VICTIMA y ACUSADO

In [19]:
hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID_HECHO         696 non-null    object
 1   FECHA            696 non-null    object
 2   HORA             696 non-null    int64 
 3   TIPO_DE_CALLE    696 non-null    object
 4   CALLE            696 non-null    object
 5   CRUCE            525 non-null    object
 6   LUGAR_DEL_HECHO  696 non-null    object
 7   COMUNA           696 non-null    int64 
 8   LONGITUD         696 non-null    object
 9   LATITUD          696 non-null    object
 10  PARTICIPANTES    696 non-null    object
 11  VICTIMA          696 non-null    object
 12  ACUSADO          696 non-null    object
dtypes: int64(2), object(11)
memory usage: 70.8+ KB


In [13]:
victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ID_VICTIMA    717 non-null    object
 1   ID_HECHO      717 non-null    object
 2   ROL           717 non-null    object
 3   SEXO          717 non-null    object
 4   EDAD          717 non-null    object
 5   RANGO_ETAREO  717 non-null    object
dtypes: object(6)
memory usage: 33.7+ KB


In [4]:
import pandas as pd
victimas = pd.read_csv("datasets/victimas.csv")
hechos = pd.read_csv("datasets/hechos.csv")

In [20]:
df_victimas = pd.merge(victimas, hechos, on='ID_HECHO', how='left')

In [21]:
df_victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID_VICTIMA       717 non-null    object
 1   ID_HECHO         717 non-null    object
 2   ROL              717 non-null    object
 3   SEXO             717 non-null    object
 4   EDAD             717 non-null    object
 5   RANGO_ETAREO     717 non-null    object
 6   FECHA            717 non-null    object
 7   HORA             717 non-null    int64 
 8   TIPO_DE_CALLE    717 non-null    object
 9   CALLE            717 non-null    object
 10  CRUCE            540 non-null    object
 11  LUGAR_DEL_HECHO  717 non-null    object
 12  COMUNA           717 non-null    int64 
 13  LONGITUD         717 non-null    object
 14  LATITUD          717 non-null    object
 15  PARTICIPANTES    717 non-null    object
 16  VICTIMA          717 non-null    object
 17  ACUSADO          717 non-null    ob

In [22]:
del df_victimas["PARTICIPANTES"]
del df_victimas["LUGAR_DEL_HECHO"]

In [23]:
df_victimas.to_csv("datasets/df_victimas.csv")